# Launch a redshift cluster and create an IAM role that has read access to S3

In [ ]:
import boto3
import configparser
import json
import pandas as pd

The IAM user with admin rights has been created in my AWS account.  Access key and secret need to be added to dwh.cfg

## Load Datawarehour parameters from the configuration file

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

DB_NAME                = config.get("DWH", "DB_NAME")
DB_USER                = config.get("DWH", "DB_USER")
DB_PASSWORD            = config.get("DWH", "DB_PASSWORD")
DB_PORT                = config.get("DWH", "DB_PORT")

## Create the IAM role

In [ ]:
# First create the IAM client
iam = boto3.client('iam', 
                   region_name='us-west-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )

try:
    print('Creating a new IAM Role')
    dwhRole = iam.create_role(
                     Path='/',
                     RoleName=DWH_IAM_ROLE_NAME,
                     Description="Allows Redshift clusters to call AWS services on your behalf",
                     AssumeRolePolicyDocument=json.dumps(
                         {'Statement': [{'Action': ['sts:AssumeRole'],  
                                         'Effect': 'Allow',
                                         'Principal': {'Service': 'redshift.amazonaws.com'}}],
                         'Version': '2012-10-17'})
                     )

except Exception as e:
    print(e)
    
# Attach Read Only policy to the IAM role
try:
    print('Attaching Policy')
    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                           PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                          )['ResponseMetadata']['HTTPStatusCode']
except Exception as e:
    print(e)
    
# Get the IAM role to be used in the next step:
print('Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

## Create the Redshift Cluster

In [ ]:
# First create the client
redshift = boto3.client('redshift',
                       region_name='us-west-2',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )
# Create the cluster:
try:
    response = redshift.create_cluster(        
        # Add parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # Parameters for identifiers & credentials
        DBName=DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD, 
        
        # Parameter for role (to allow s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

## Run this piece of code until the cluster appears as available

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

## Extract the cluster endpoint and role ARN to connect to the database

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

## Open an incoming TCP port to connect to the cluster

In [ ]:
ec2 = boto3.resource('ec2',
                   region_name='us-west-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )

try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

## Delete IAM role and cluster

In [ ]:
# First create the client
redshift = boto3.client('redshift',
                       region_name='us-west-2',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)


In [ ]:
# First create the IAM client
iam = boto3.client('iam', 
                   region_name='us-west-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                  )
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)

## Explore the s3 bucket

In [ ]:
s3 = boto3.resource('s3',
                  region_name='us-west-2',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET
                 )
sparkifyBucket = s3.Bucket('udacity-dend')

In [ ]:
for obj in sparkifyBucket.objects.filter(Prefix='log-data'):
    #key = obj.key
    #print(key)
    body = obj.get()['Body'].read()
    print(body.decode("utf-8"))

In [ ]:
sparkifyBucket = s3.Bucket('udacity-dend')
for obj in sparkifyBucket.objects.filter(Prefix='log_json_path.json'):
    key = obj.key
    print(key)
    body = obj.get()['Body'].read()
    print(body.decode("utf-8"))

In [ ]:
objects = sparkifyBucket.objects.filter(Prefix='song_data')


In [ ]:
for obj in objects:
    key = obj.key
    print(key)
    body = obj.get()['Body'].read().decode("utf-8")
    if "Clocks" in body:
        print(body)